In [1]:
import feather
import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
import theano.tensor as tt

import theano


%matplotlib inline
sns.set_context('paper')
sns.set_style('darkgrid')

path = '/Users/chirag/Desktop/pymc_data.feather'
df = feather.read_dataframe(path)



df1 = df[['pheno_T1_1', 'pheno_T2_1']]

data=df1


D = 2
K = 2


In [2]:
def n_triu(d):
    return (d-1)*d/2

def n_triu_inv(t, module=np):
    return int(1/2*(1 + module.sqrt(1 + 8*t)))

def make_cov_matrix(sigmas, C_triu, module=np):
    D = len(sigmas.tag.test_value if hasattr(sigmas, "tag") else sigmas)
    C = module.ones((D,D))
    Sigmas = module.diag(sigmas)
    # indices are always numpy
    idxu = np.triu_indices(D, 1)
    idxl = np.tril_indices(D, -1)
    if module.__name__ == "theano.tensor":
        C = tt.set_subtensor(C[idxu], C_triu)
        C = tt.set_subtensor(C[idxl], C_triu)
    elif module.__name__ == "numpy":
        C[idxu] = C_triu
        C[idxl] = C_triu
    return Sigmas.dot(C).dot(Sigmas)


In [3]:
init_mean = np.mean(data, axis=0)
init_sigma = np.std(data, axis=0).astype(np.float32) / 10
idxu = np.triu_indices(D, 1)
init_corr = np.corrcoef(data, rowvar=0)[idxu].astype(np.float32)


In [4]:
def multivariatenormal(init_mean, init_sigma, init_corr, suffix="", dist=False):
    if not isinstance(suffix, str):
        suffix = str(suffix)
    D = len(init_sigma)
    
    sigma = pm.Lognormal('sigma' + suffix, np.zeros(D, dtype=np.float), np.ones(D), shape=D, testval=init_sigma)
    nu = pm.Uniform('nu' + suffix, 0, 5)
    C_triu = pm.LKJCorr('C_triu' + suffix, nu, D, testval=init_corr)
    cov = pm.Deterministic('cov' + suffix, make_cov_matrix(sigma, C_triu, module=tt))
    
    mu = pm.MvNormal('mu' + suffix, 0, cov, shape=2, testval=init_mean)

    return pm.MvNormal.dist(mu, cov) if dist else pm.MvNormal('mvn' + suffix, mu, cov)



In [5]:

with pm.Model() as model:
    #alpha = pm.Gamma('alpha', 1., 1.)
    #beta = pm.Beta('beta', 1, alpha, shape=K)
    #w = pm.Deterministic('w', stick_breaking(beta))
    w = pm.Dirichlet('w', np.ones(K))
    obs = pm.Mixture('obs', w, [multivariatenormal(init_mean, init_sigma, init_corr, k, dist=True) for k in range(K)], observed=data)


In [7]:
with model:
    trace = pm.sample(1000, pm.Metropolis())[500::5]


AsTensorError: ('Variable type field must be a TensorType.', mu0_shared, <theano.gof.type.Generic object at 0x111f763c8>)